In [10]:
#連接google雲端
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
files.upload()

In [25]:
# !rm -rf /content/drive/MyDrive/Colab

In [ ]:
# !7z x ng.7z

In [13]:
# !find /content -name 'yolov4-tiny-obj_last (1).weights'

/content/Colab/darknet/yolov4-tiny-obj_last (1).weights


In [ ]:
# %cd /content/drive/MyDrive/
# %cp -r /content/drive/MyDrive/Colab/ ./
%cp -r /content/Colab/ /content/drive/MyDrive/
# %cd /content/

In [ ]:
# !pip install opencv-python==4.2.0.32

In [ ]:
# %cd darknet
# !sed -i '212s/255/18/' darknet/cfg/yolov4-tiny-obj.cfg
# !sed -i '220s/80/1/' darknet/cfg/yolov4-tiny-obj.cfg
# !sed -i '263s/255/18/' darknet/cfg/yolov4-tiny-obj.cfg
# !sed -i '269s/80/1/' darknet/cfg/yolov4-tiny-obj.cfg
# !make
# !./darknet detector calc_anchors ../Nozzle_detection/cfg/nozzle.data -num_of_clusters 6 -width 416 -height 416 -showpause
# !./darknet detector train ../Nozzle_detection/cfg/nozzle.data cfg/yolov4-tiny-obj.cfg ../Nozzle_detection/cfg/yolov4-tiny.conv.29 -dont_show
# !./darknet detector test ../Nozzle_detection/cfg/nozzle.data cfg/yolov4-tiny-obj.cfg ../Nozzle_detection/weights/yolov4-tiny-obj_last.weights ../ng/nozzle_ng196.jpg
# !./darknet detector map ../Nozzle_detection/cfg/nozzle.data cfg/yolov4-tiny-obj.cfg ../Nozzle_detection/weights/yolov4-tiny-obj_last.weights
# !./darknet detector recall  ../Nozzle_detection/cfg/nozzle.data cfg/yolov4-tiny-obj.cfg ../Nozzle_detection/weights/yolov4-tiny-obj_last.weights


In [19]:
import os
import shutil
import cv2

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def run_convert(data_file, wider_train, yolo_path, file_info_name, write_txt):
    now_path = os.getcwd()
    data_counter = 0

    with open(data_file, 'r') as f:
        print("read file...")
        data = f.readlines()
        
        for data_line in data:
            data_line = data_line.strip()
            data_info = data_line.split('/')
            # sub_count = 0
            # the image name
            if len(data_info) == 2:
                label_0_counter = 1
                data_info_path = os.path.join(data_info[0], data_info[1])
                data_path = os.path.join(wider_train, data_info_path)

                # copy image to yolo path and rename
                shutil.copyfile(data_path, yolo_path + str(data_counter) + '.jpg')
                
                image = cv2.imread(data_path)
                image_size = [image.shape[1],image.shape[0]]
                
                # image --> rename
                with open(file_info_name, 'a') as f:
                    line_txt = [data_info_path, ' --> ', yolo_path + str(data_counter) + '.jpg', '\n']
                    f.writelines(line_txt)

                with open(write_txt, 'a') as f:
                    path = os.path.join(now_path, yolo_path)
                    line_txt = [path + str(data_counter) + '.jpg', '\n']
                    f.writelines(line_txt)
                
                data_counter += 1
                label_list = []
                # process other info
                sub_count = 1
                continue

            # the count of bndBox
            if sub_count == 1:
                sub_count += 1
                continue

            # bndBox info
            # print("process ", label_0_counter, " bndBox info...")
            if sub_count >= 2:
                label_0_counter += 1
                info_list = data_line.split(' ')
#                 print("WIDER FACE(x1, y1, w, h): ", info_list[0], info_list[1], info_list[2], info_list[3])
                
                xmin = int(info_list[0])
                xmax = int(info_list[0])+int(info_list[2])
                ymin = int(info_list[1])
                ymax = int(info_list[1])+int(info_list[3])
                
                box = [xmin, xmax, ymin, ymax]
                x, y, w, h = convert(image_size,box)
#                 print("YOLO txt(x, y, w, h): ", x, y, w, h)
                
                with open(yolo_path + str(data_counter-1) + '.txt', 'a+') as f:
                    f.write('0 %s %s %s %s\n' % (x, y, w, h))
                    
    print('the file is processed')


wider_train = "img_0520"
yolo_path = "Nozzle_detection/yolo_train/"
data_file = "Nozzle_detection/nozzle_train_bbx_gt.txt"
file_info_name = 'file_info_train.txt'
write_txt = 'Nozzle_detection/cfg/train.txt'

# wider_train = "WIDER_val/images"
# yolo_path = "yolo_val/"
# data_file = "wider_face_split/wider_face_val_bbx_gt.txt"
# file_info_name = 'file_info_val.txt'
# write_txt = 'cfg/val.txt'

if not os.path.exists(yolo_path):
    os.mkdir(yolo_path)
else:
    lsdir = os.listdir(yolo_path)
    for name in lsdir:
        if name.endswith('.txt') or name.endswith('.jpg'):
            os.remove(os.path.join(yolo_path, name))

if os.path.exists(file_info_name):
    file=open(file_info_name, 'w')
    
if os.path.exists(write_txt):
    file=open(write_txt, 'w')

run_convert(data_file, wider_train, yolo_path, file_info_name, write_txt)

read file...
the file is processed


In [12]:
import cv2
import os
import numpy as np

weightsPath = "Nozzle_detection/weights/yolov4-tiny-obj_last.weights"
configPath = "darknet/cfg/yolov4-tiny-obj.cfg"
labelsPath = "Nozzle_detection/cfg/nozzle.names"
imgdir = "img_0520"
save_result_dir = "detect_result"
save_txt_dir = "detect_txt"

if not os.path.exists(save_result_dir):
    os.mkdir(save_result_dir)
    
if not os.path.exists(save_txt_dir):
    os.mkdir(save_txt_dir)    

LABELS = open(labelsPath).read().strip().split("\n")
# Set according to the number of categories
COLORS = [(0, 255, 255), (255, 255, 0), (0, 255, 0), (255, 0, 0)]

CONFIDENCE_THRESHOLD = 0.7
NMS_THRESHOLD = 0.4

net = cv2.dnn.readNet(weightsPath, configPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

model = cv2.dnn_DetectionModel(net)
# Set according to the size of input
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

for image_name in os.listdir(imgdir):
    print("detect " + image_name + " ...")
    name = image_name.split('.jpg')[0]

    img = cv2.imread(os.path.join(imgdir, image_name))
    image_size = [img.shape[1],img.shape[0]]
    
    classes, scores, boxes = model.detect(img, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    print(classes, scores, boxes)
    '''
    for (classid, score, box) in zip(classes, scores, boxes):
        color = COLORS[int(classid) % len(COLORS)]
        label = "%s" % (LABELS[classid[0]])
        
        xmin = int(box[0])
        xmax = int(box[0])+int(box[2])
        ymin = int(box[1])
        ymax = int(box[1])+int(box[3])
        
        bndbox = [xmin, xmax, ymin, ymax]
        x, y, w, h = convert(image_size, bndbox)
 
        cv2.rectangle(img, box, color, 2)
        text = label+ ":"+str(score[0])
        cv2.putText(img, text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        with open(os.path.join(save_txt_dir, name+'.txt'), 'a+') as f:
            # yolo txt
            f.write('%s %s %s %s %s\n' % (classid[0], x, y, w, h))
        
    cv2.imwrite(os.path.join(save_result_dir, name+".jpg"), img)
    '''

detect 210600_cam.jpg ...
() () ()
detect 020000_cam.jpg ...
() () ()
detect 132400_cam.jpg ...
() () ()
detect 121200_cam.jpg ...
() () ()
detect 141200_cam.jpg ...
() () ()
detect 233600_cam.jpg ...
() () ()
detect 015400_cam.jpg ...
() () ()
detect 123000_cam.jpg ...
() () ()
detect 171800_cam.jpg ...
() () ()
detect 003630_cam.jpg ...
() () ()
detect 050600_cam.jpg ...
() () ()
detect 171200_cam.jpg ...
() () ()
detect 083000_cam.jpg ...
() () ()
detect 234200_cam.jpg ...
() () ()
detect 090000_cam.jpg ...
() () ()
detect 074200_cam.jpg ...
() () ()
detect 053600_cam.jpg ...
() () ()
detect 200600_cam.jpg ...
() () ()
detect 034800_cam.jpg ...
() () ()
detect 033000_cam.jpg ...
() () ()
detect 211800_cam.jpg ...
() () ()
detect 202400_cam.jpg ...
() () ()
detect 061200_cam.jpg ...
() () ()
detect 054800_cam.jpg ...
() () ()
detect 140000_cam.jpg ...
() () ()
detect 100000_cam.jpg ...
() () ()
detect 022400_cam.jpg ...


KeyboardInterrupt: ignored